# EE 467 Homework 1: Twitter Spam Detection

Welcome to the first homework of EE 467! Here we will apply the ML pipline introduced in the second lab to **Twitter spam message detection problems**. In the past few years, social medias like Twitter has become the target of spam posting. Automatic spam posts can not only affect user experience, but they can also spread misinformation and subtly affect public opinions.

In this homework, we will work on a mini Twitter message dataset extracted from the CRESCI-2017 dataset. This dataset contains 50000 Twitter messages, half of which are sent by real Twitter users and half of which generated by spam bots. Your task is to train classifiers that can tell these two kinds of messages apart. This time, you'll go deeper and try multiple feature extractors and classifiers. Like the labs, you will need to fill out blanks in code cells. You are also going to answer a few questions and **optionally** work on improvements to existing pipeline completely on your own.

## Before You Start

1. You can discuss on Slack if you have questions and want to seek help; however, please try your best to **limit the scope of your question** and **avoid asking directly for answers**. You should also **avoid copy-pasting answers and code** from others.
2. You **may use AI assistants** (e.g., ChatGPT, Gemini, etc.) to support your work. If you do, you must **clearly acknowledge** this by stating:

- **The name of the AI assistant**, and  
- **How it was used** (e.g., brainstorming, clarifying concepts, debugging, editing, generating examples)

You **must not copy and paste** AI-generated content directly into your submission without **fully understanding** what it does and being able to **explain it in your own words**.

3. You can **optionally work on this homework with one other student of this course as a team**, but **each of you needs to submit the homework individually**. If you choose to form a group, please include the name of your teammate here: Nate Snyder.

## Feature Extraction

We will begin our ML pipeline with feature extraction in this homework, as the dataset is good enough and does not need any special pre-processing. As usual, we will load the dataset into memory, extract samples features into vectors and split the dataset into training and test sets.

In [1]:
import pandas as pd

data = pd.read_csv('./twitter-mini.csv')
print(data.head(5))

                                                text  label
0  @baydiangirl it definitely moved some church f...      0
1  @suebob Looks like #TXlege made up new rules f...      0
2  RT @saleemchat: #WJChat #A1B 5 yrs: People ski...      0
3                            @SajidaBalouch Huhhhhh       0
4              @Dukester_94 im the lad of all bibles      0


In lab 2 we have learned how to use the `CountVectorizer` feature extractor, which counts the occurance of each token (letter here) in a sample (Twitter message text here). This time, we will also try `TfidfVectorizer` and see which feature extractor performs better. Here we will introduce the algorithm it implemented, **TF-IDF**:

> **TF-IDF** is the abbreviation for **term frequency–inverse document frequency**, and is defined as the product of the two frequency. The most classical version of TF-IDF is:
>
> $$
tf-idf(t) = tf(t, d) \cdot idf(t, D) = \frac{f_{t,d}}{|d|} \log \frac{|D|}{|D_t|}
$$
>
> Where $t$ is the term, $D$ is the collection of all documents, $d$ is one of the document that contains $t$ and $D_t$ is the subset of documents that contains term $t$. The advantage of TF-IDF is that frequent and useless words, such as "the", "a" and "an" will receive lower weight, since they appear in almost all documents. On the other hand, unique yet frequent (in any $d \in D_t$) words will get higher weight. Research has shown that TF-IDF is easy to implement and performs relatively well.

Before we run these feature extractors on our dataset, we need to convert each message into a sequence of terms. Since Twitter messages often contain emojis, abbreviations, hashtags and other composition of characters, we can't use the word-level analyzer as we have done in lab 2. Instead, we will generate **character N-grams** from these messages.

> A **character N-gram** consists of N consecutive characters. Here we show all character 1, 2, 3 and 4-grams for word "cold":
> <img src="attachment:char-ngram.jpg" width="50%" />

Both `CountVectorizer` and `TfidfVectorizer` provide support for character N-grams. Refer to `sklearn`'s documents to build a character N-gram analyzer.

In [4]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# 1) Extract features of all messages into `feat1` with `CountVectorizer` using CHARACTER trigrams
print("Extracting features with `CountVectorizer`...")
feat1 = CountVectorizer(ngram_range=(3, 3)).fit_transform(data["text"])

# 2) Repeat with `TfidfVectorizer` and save generated features as `feat2`
print("Extracting features with `TfidfVectorizer`...")
feat2 = TfidfVectorizer(ngram_range=(3, 3)).fit_transform(data["text"])

print("Completed.")

Extracting features with `CountVectorizer`...
Extracting features with `TfidfVectorizer`...
Completed.


We will also try combining two kinds of feature generated above into a single feature. We will use `scipy.sparse.hstack` to concatenate corresonding features for each sample. Remember, concatenated features matrix should have same amount of samples as `feat1` and `feat2`.

In [5]:
from scipy.sparse import hstack
# Concatenate token count features and TF-IDF features on the feature dimension,
# name the combined feature matrix `feat_comb`.
feat_comb = hstack((feat1, feat2))

Finally, let's take a look at the labels. In our dataset, 0 represents messages from real Twitter users and 1 represents spam messages.

In [6]:
y = data["label"].values
print(y)

[0 0 0 ... 1 1 1]


## Training

As usual, we will start by doing a 80% / 20% training-test split. Like lab 2, we suggest you to **use the same random seed for all splits** for comparable and reproducible results.

In [7]:
from sklearn.model_selection import train_test_split

TVT_SPLIT_SEED = 42
TEST_PROPORTION = 0.2

# 1) Split token count features into 80/20 training and test sets
# Token count features
print("Spliting token count features...")
feat1_tv, feat1_test, feat1_label_tv, feat1_label_test = train_test_split(
    feat1, data["label"], test_size=TEST_PROPORTION, random_state=TVT_SPLIT_SEED
)

# 2) Split TF-IDF features into 80/20 training and test sets
# TF-IDF features
print("Spliting TF-IDF features...")
feat2_tv, feat2_test, feat2_label_tv, feat2_label_test = train_test_split(
    feat2, data["label"], test_size=TEST_PROPORTION, random_state=TVT_SPLIT_SEED
)

# 3) Split combined features into 80/20 training and test sets
# Combined features
print("Spliting combined features...")
feat_comb_tv, feat_comb_test, feat_comb_label_tv, feat_comb_label_test = train_test_split(
    feat_comb, data["label"], test_size=TEST_PROPORTION, random_state=TVT_SPLIT_SEED
)

print("Completed.")

Spliting token count features...
Spliting TF-IDF features...
Spliting combined features...
Completed.


We are now ready to train **binary classifiers** for spam message detection. Apart from the `LogisticRegression` classifier you have already seen and used during Lab 1, we will try a few more classifiers:

* `MultinomialNB` (Naive Bayes):  
  This classifier is based on the **Naive Bayes** assumption that, *given the class label*, individual features are conditionally independent.  
  For text classification, `MultinomialNB` is used with **token count** or **TF-IDF-like** features.

  It models the probability of a message belonging to a class $ y \in \{\text{spam}, \text{ham}\} $ as:

  $$
  P(y \mid \mathbf{x}) \propto P(y)\prod_{j=1}^{d} P(x_j \mid y)
   $$

  where $\mathbf{x} = (x_1, x_2, \dots, x_d)$ is the feature vector (e.g., word counts), and $P(x_j \mid y)$ is learned from training data.  
  The predicted label is chosen using:

   $$
  y_{pred} = \arg\max_y \; P(y \mid \mathbf{x})
   $$

  In practice, the computation is done in the **log domain** to avoid numerical underflow:

  $$
  y_{pred} = \arg\max_y \left( \log P(y) + \sum_{j=1}^{d} x_j \log P(x_j \mid y) \right)
  $$

* `LinearSVC`:  
  This classifier is based on **linear SVM**, a popular algorithm for binary classification. The linear SVM algorithm treats the feature vector of each sample as a point in a linear space, and tries to find a hyperplane that separates points of the two classes as well as possible.  

  In practice, we optimize a **squared hinge loss with regularization**, since the features may not be perfectly linearly separable:

   $$
  l_{hinge} = \max(0, y(1 - \mathbf{x}^T \mathbf{w}))^2 + \beta ||\mathbf{w}||_2^2
   $$

  Similarly, the classification result is determined by:

  $$
  y_{pred} = sign(1 - \mathbf{x}^T \mathbf{w})
 $$

We have three kinds of features (**token count**, **TF-IDF**, and **combined**) and three kinds of classifiers. Here we will only try three combinations and compare their performance:

* `MultinomialNB` on token count features  
* `LogisticRegression` on TF-IDF features  
  - Try increasing the number of iterations if a warning of non-convergence pops up  
* `LinearSVC` on combined features  
  - For linear SVM, you can ignore the non-convergence warning as it does not affect performance  


In [9]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

# 1) Train a `MultinomialNB` classifier called `model1` on token count training set
print("Training Naive Bayes classifier on token count training set...")
model1 = MultinomialNB()
model1.fit(feat1_tv, feat1_label_tv)

# 2) Train a `LogisticRegression` classifier called `model2` on TF-IDF training set
print("Training logistic regression classifier on TF-IDF training set...")
model2 = LogisticRegression(max_iter=1000, random_state=0)
model2.fit(feat2_tv, feat2_label_tv)

# 3) Train a `LinearSVC` classifier called `model_comb` on combined features training set
print("Training SVM classifier on combined features training set...")
model_comb = LinearSVC(random_state=0)
model_comb.fit(feat_comb_tv, feat_comb_label_tv)

print("Completed.")

Training Naive Bayes classifier on token count training set...
Training logistic regression classifier on TF-IDF training set...
Training SVM classifier on combined features training set...
Completed.


## Evaluation

Finally, it's time to evaluate our models. You probably noticed that we have repeated similar code for different feature extraction and classification models. To avoid that, we will use `for` loops to make evaluation simpler:

In [10]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

features = [("Token count", feat1), ("TF-IDF", feat2), ("Combined", feat_comb)]
models = [("Naive Bayes", model1), ("Logistic", model2), ("SVM", model_comb)]

# Complete missing code in the loop body
for i, ((feat_name, feat), (model_name, model)) in enumerate(zip(features, models)):
    # Begin of metrics for combination
    print("====================")
    print(f"{i+1}. {model_name} on {feat_name}:\n")

    # 1) Split features into training and test set
    # (Note: you should use the same random number seed as the one in the training-test split above)
    tv, test, label_tv, label_test = train_test_split(
        feat, data["label"], test_size=TEST_PROPORTION, random_state=TVT_SPLIT_SEED
    )

    # 2) Predict labels for training data
    pred_train = model.predict(tv)

    # 3) Print metrics for training set (precision, recall and F1 metrics)
    print("Training metrics:", classification_report(label_tv, pred_train))
    # Confusion matrix
    print("Training confusion matrix:\n", confusion_matrix(label_tv, pred_train), "\n")
    # Accuracy
    print("Train accuracy:", accuracy_score(label_tv, pred_train), "\n")

    # 4) Predict labels for test data
    pred_test = model.predict(test)

    # 5) Print metrics for test set (precision, recall and F1 metrics)
    print("Test metrics:", classification_report(label_test, pred_test))
    # Confusion matrix
    print("Test Confusion Matrix:\n", confusion_matrix(label_test, pred_test), "\n")
    # Accuracy
    print("Test Accuracy:", accuracy_score(label_test, pred_test))

    # End of metrics for combination
    print("====================\n")

1. Naive Bayes on Token count:

Training metrics:               precision    recall  f1-score   support

           0       1.00      0.93      0.97     19978
           1       0.94      1.00      0.97     20022

    accuracy                           0.97     40000
   macro avg       0.97      0.97      0.97     40000
weighted avg       0.97      0.97      0.97     40000

Training confusion matrix:
 [[18646  1332]
 [    9 20013]] 

Train accuracy: 0.966475 

Test metrics:               precision    recall  f1-score   support

           0       0.82      0.91      0.86      5022
           1       0.90      0.80      0.85      4978

    accuracy                           0.85     10000
   macro avg       0.86      0.85      0.85     10000
weighted avg       0.86      0.85      0.85     10000

Test Confusion Matrix:
 [[4569  453]
 [ 997 3981]] 

Test Accuracy: 0.855

2. Logistic on TF-IDF:

Training metrics:               precision    recall  f1-score   support

           0       0.7

# Questions
Please answer the following questions. You can just write down the answer below each question in this cell.


1. Please compute `tf-idf` of word-level **bigrams** for the following three messages **by hand**. You should use the TF-IDF definition introduced in class.

  * Message 1: "vegetables are good for health"
  * Message 2: "fruits are good source of vitamins"
  * Message 3: "proteins are good for health"

  Compute and sort intermediate and final results for **all occurring bigrams in the following order**:
  
  ```
  ["vegetables are", "are good", "good for", "for health", "fruits are", "good source", "source of", "of vitamins", "proteins are"]
  ```
  For all messages:
  * IDF computation:
    * Number of total documents: 3
    * Counts of occurances for each bigram: $\begin{bmatrix} 1 & 3 & 2 & 2 & 1 & 1 & 1 & 1 & 1 \end{bmatrix}$
    * $\log_{10}(3 / \begin{bmatrix} 1 & 3 & 2 & 2 & 1 & 1 & 1 & 1 & 1 \end{bmatrix})$
    * IDF = $\begin{bmatrix} 0.47712125472 & 0 & 0.176091259 & 0.176091259 & 0.47712125472 & 0.47712125472 & 0.47712125472 & 0.47712125472 & 0.47712125472 \end{bmatrix}$

  Message 1:
  * Term Occurance: $\begin{bmatrix} 1 & 1 & 1 & 1 & 0 & 0 & 0 & 0 & 0 \end{bmatrix}$
  * TF = $\begin{bmatrix} .25 & .25 & .25 & .25 & 0 & 0 & 0 & 0 & 0 \end{bmatrix}$
  * TF-IDF = $\begin{bmatrix} .25 & .25 & .25 & .25 & 0 & 0 & 0 & 0 & 0 \end{bmatrix} \odot \begin{bmatrix} 0.47712125472 & 0 & 0.176091259 & 0.176091259 & 0.47712125472 & 0.47712125472 & 0.47712125472 & 0.47712125472 & 0.47712125472 \end{bmatrix}$
  * TF-IDF = $\begin{bmatrix} 0.119280314 & 0 & 0.0440228147 & 0.0440228147 & 0 & 0 & 0 & 0 & 0 \end{bmatrix}$

  Message 2:
  * Term Occurance: $\begin{bmatrix} 0 & 1 & 0 & 0 & 1 & 1 & 1 & 1 & 0 \end{bmatrix}$
  * TF = $\begin{bmatrix} 0 & .2 & 0 & 0 & .2 & .2 & .2 & .2 & 0 \end{bmatrix}$
  * TF-IDF = $\begin{bmatrix} 0 & .2 & 0 & 0 & .2 & .2 & .2 & .2 & 0 \end{bmatrix} \odot \begin{bmatrix} 0.47712125472 & 0 & 0.176091259 & 0.176091259 & 0.47712125472 & 0.47712125472 & 0.47712125472 & 0.47712125472 & 0.47712125472 \end{bmatrix}$
  * TF-IDF = $\begin{bmatrix} 0 & 0 & 0 & 0 & 0.0954242509 & 0.0954242509 & 0.0954242509 & 0.0954242509 & 0 \end{bmatrix}$


  Message 3:
  * Term Occurance: $\begin{bmatrix} 0 & 1 & 1 & 1 & 0 & 0 & 0 & 0 & 1 \end{bmatrix}$
  * TF = $\begin{bmatrix} 0 & .25 & .25 & .25 & 0 & 0 & 0 & 0 & .25 \end{bmatrix}$
  * TF-IDF = $\begin{bmatrix} 0 & .25 & .25 & .25 & 0 & 0 & 0 & 0 & .25 \end{bmatrix} \odot \begin{bmatrix} 0.47712125472 & 0 & 0.176091259 & 0.176091259 & 0.47712125472 & 0.47712125472 & 0.47712125472 & 0.47712125472 & 0.47712125472 \end{bmatrix}$
  * TF-IDF = $\begin{bmatrix} 0 & 0 & 0.0440228147 & 0.0440228147 & 0 & 0 & 0 & 0 & 0.119280314 \end{bmatrix}$
  
  Then complete the following questions:

  1. Term Frequency (`tf`) of **bigrams for message 2**.  TF = $\begin{bmatrix} 0 & .2 & 0 & 0 & .2 & .2 & .2 & .2 & 0 \end{bmatrix}$
  2. Inverse Document Frequency (`idf`) of **bigrams for message 2**.  IDF = $\begin{bmatrix} 0.47712125472 & 0 & 0.176091259 & 0.176091259 & 0.47712125472 & 0.47712125472 & 0.47712125472 & 0.47712125472 & 0.47712125472 \end{bmatrix}$
  3. `tf-idf` of **bigrams for message 2**.  TF-IDF = $\begin{bmatrix} 0 & 0 & 0 & 0 & 0.0954242509 & 0.0954242509 & 0.0954242509 & 0.0954242509 & 0 \end{bmatrix}$
  4. Is the `tf-idf` value for the term "are good" equal to 0? If it's 0, can you explain why it makes sense to have 0 for that term?  _Yes it is zero.  It makes sense because "are good" is in every document, so it being included in message 2 does not make in any different from the rest._
  5. If we consider unigram (1-gram) based bag of words, how many words in these three messages would be assigned a 0 value tf-idf? And please explain why.
_The IDF array would have only two zeros, at position 2 and 3 for "are" and "good".  The total number of unique words is 10.  For the TF-IDF for each message, it is easier to count the number of non-zero elements.  For message 1, there would be 3 non-zero elements.  Because the words "are" and "good" correspond to 0 in the IDF vector, and the rest of the words are non-zero in the IDF vector.  So there are 7 elements that are zero.  For message 2, there would be 4 non-zero elements.  Because the words "are" and "good" correspond to 0 in the IDF vector, and the rest of the words are non-zero in the IDF vector, so there are 6 elements that are zero.  For message 3, there would be 3 non-zero elements.  Because the words "are" and "good" correspond to 0 in the IDF vector, and the rest of the words are non-zero in the IDF vector, so there are 7 elements that are zero._


2. What does `support` mean in the `classification_report`?

It is the number of data points that correspond to the label.

3. If the training data is highly unbalanced, is `accuracy` a good metric for the classifiers' performance? Does a high accuracy necessarily means a high f1-score? Compare `accuracy` with `f1-score`. Based on your criterion, which type of classifier is better? Please give some justifications to support your answer.

If the dataset is unbalanced, accuracy is not good.  It will unfairly weigh the data points that are overrepresented with much higher weight, and the ones where the data poitns are underrepresented with much lower weight.  So it could seem accurate, but in reality, one of the labels is not being accuratly predicted at all.

f1-score is much better.  Accuracy is a simple ratio of correct labels to total number of labels.  F1-score includes precision and recall, which is a better metric for how well a model performs for the different labels.  If a purely accuracy based system were used, then a data set where a certain label were barely tested would not be represented properly.  For example, if I train a model then test it and it has an accuracy of .99, but always get's one of the labels wrong, it would still appear good, but the F1-score would properly reflect a lower score.

4. For most cases, the amount of bad data is much less than good data. How can you mitigate the data imbalance issue?

You can simply find more cases of the bad data.  Or you can reduce the amount of good data.  There isn't an easy solution, and gathering examples of both is required to make a good model.

## Model Improvements (Optional; Not Included in Homework Score)

As we have pointed out in the first class, machine learning pipeline does not end with a single iteration of training, testing and evaluation. Here, you will repeat these process with different hyper-parameters and try to improve the spam detection model so that it performs even better.

Below are two **potential improvements** that may be beneficial to the performance. You are encouraged to complete either (or both) of them. Your response to these questions are optional and won't be graded, but we will provide feedbacks to your code and analysis if you choose to answer them. You can add code and text cells below the questions as needed.

1. We use character-level trigrams as tokens during the feature extraction stage, however $N = 3$ is a rather arbitrary choice. What will the performance be like if we use shorter N-grams, such as $N = 1$ and $N = 2$? What about longer N-grams, such as $N = 4$? What if we combine different N-grams, like $N = 1, 2, 3$? Modify the feature extraction code above and play with different (range of) $N$ values. What are **the best (range of) $N$ value on the test set of combined features and SVM classifier**? What metrics did you use to reach this conclusion and why do you think this (range of) $N$ value is the best?

2. During the above training and evaluation process, we are only evaluating three combination of feature extractors and classifiers. Can you try to rewrite the training and evaluation code, such that **every combination** of feature extractors and classifiers can be evaluated? What metrics did you use to reach this conclusion and what is the best combination of feature extractor and classifier in terms of test set performance?

## References

1. CRESCI-2017 Dataset: https://botometer.osome.iu.edu/bot-repository/datasets.html
2. TF-IDF: https://en.wikipedia.org/wiki/Tf%E2%80%93idf
3. N-grams: https://en.wikipedia.org/wiki/N-gram
4. `scikit-learn` documentation: https://scikit-learn.org/stable/modules/classes.html
5. Ridge regression: https://en.wikipedia.org/wiki/Tikhonov_regularization
6. SVM: https://en.wikipedia.org/wiki/Support-vector_machine
7. Lasso: https://en.wikipedia.org/wiki/Lasso_(statistics)
8. ElasticNet: https://en.wikipedia.org/wiki/Elastic_net_regularization
9. Kernel method: https://en.wikipedia.org/wiki/Kernel_method